In [25]:
import os
import cv2
import numpy as np
import string
import time
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

import tensorflow.keras.backend as K
from tensorflow.keras.preprocessing.sequence import pad_sequences
# from tensorflow.keras.utils import multi_gpu_model
from tensorflow.keras.layers import Dense, Reshape, BatchNormalization, Input, Conv2D, MaxPool2D, Lambda, Bidirectional

from tensorflow.compat.v1.keras.layers import CuDNNLSTM
from tensorflow.keras.models import Model
from tensorflow.keras.activations import relu, sigmoid, softmax

from tensorflow.keras.utils import to_categorical, Sequence
from tensorflow.keras.callbacks import ModelCheckpoint
from tqdm import tqdm
from collections import Counter

from PIL import Image

print(tf.__version__)


2.4.1


In [26]:
char_list = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

In [27]:
# input with shape of height=32 and width=128 
# inputs = Input(shape=(32, 128, 1))

# conv_1 = Conv2D(16, (3,3), activation = 'relu', padding='same')(inputs)
# pool_1 = MaxPool2D(pool_size=(2, 2), strides=2)(conv_1)
 
# conv_2 = Conv2D(32, (3,3), activation = 'relu', padding='same')(pool_1)
# pool_2 = MaxPool2D(pool_size=(2, 2), strides=2)(conv_2)

# conv_3 = Conv2D(64, (3,3), activation = 'relu', padding='same')(pool_2)
# conv_4 = Conv2D(64, (3,3), activation = 'relu', padding='same')(conv_3)
# pool_4 = MaxPool2D(pool_size=(2, 1))(conv_4)
 
# conv_5 = Conv2D(64, (3,3), activation = 'relu', padding='same')(pool_4)
# # Batch normalization layer
# batch_norm_5 = BatchNormalization()(conv_5)
 
# conv_6 = Conv2D(64, (3,3), activation = 'relu', padding='same')(batch_norm_5)
# batch_norm_6 = BatchNormalization()(conv_6)
# pool_6 = MaxPool2D(pool_size=(2, 1))(batch_norm_6)
 
# conv_7 = Conv2D(64, (2,2), activation = 'relu')(pool_6)
 
# squeezed = Lambda(lambda x: K.squeeze(x, 1))(conv_7)
 
# # bidirectional LSTM layers with units=128
# blstm_1 = Bidirectional(CuDNNLSTM(128, return_sequences=True))(squeezed)
# blstm_2 = Bidirectional(CuDNNLSTM(128, return_sequences=True))(blstm_1)

# outputs = Dense(len(char_list) + 1, activation = 'softmax')(blstm_2)

# # model to be used at test time
# model = Model(inputs, outputs)

In [28]:
max_label_len = 23

labels = Input(name='the_labels', shape=[max_label_len], dtype='float32')
input_length = Input(name='input_length', shape=[1], dtype='int64')
label_length = Input(name='label_length', shape=[1], dtype='int64')

def ctc_lambda_func(args):
    y_pred, labels, input_length, label_length = args 
    return K.ctc_batch_cost(labels, y_pred, input_length, label_length)
 
loss_out = Lambda(ctc_lambda_func, 
                  output_shape=(1,), 
                  name='ctc')([outputs, labels, input_length, label_length])

#model to be used at training time
model = Model(inputs=[inputs, labels, input_length, label_length], outputs=loss_out)

file_path = "C_LSTM_best.hdf5"

model.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer = 'adam')


In [29]:
# model = tf.keras.models.load_model("model.h5")
# tf.saved_model.save(act_model, "C_LSTM_best.hdf5")
model.load_weights('model.h5')


# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.allow_custom_ops = True
converter.experimental_new_converter = True
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS,
                                       tf.lite.OpsSet.SELECT_TF_OPS]
tflite_model = converter.convert()

# Save the model.
with open('model.tflite', 'wb') as f:
    f.write(tflite_model)